In [0]:
import os, shutil
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [0]:
df = pd.read_csv('gdrive/My Drive/ML/Projects/Kaggle/Aptos/train.csv')

Create a dictionary with labels corresponding to their image names

In [0]:
fname_dict = {0 : [], 1 : [], 2 : [], 3 : [], 4 : []}
for index, row in df.iterrows():
  fname_dict[row['diagnosis']] += [row['id_code']]

In [0]:
original_dataset_dir = 'gdrive/My Drive/ML/Projects/Kaggle/Aptos/train_images'
dataset_dir = 'gdrive/My Drive/ML/Projects/Kaggle/Aptos'
train_dir = os.path.join(dataset_dir, 'train_dir')
os.mkdir(train_dir)
validation_dir = os.path.join(dataset_dir, 'validation_dir')
os.mkdir(validation_dir)
test_dir = os.path.join(dataset_dir, 'test_dir')
os.mkdir(test_dir)

In [0]:
import cv2
IMG_SIZE = 2000

def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
    return img

def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [0]:
train_class_dirs = [] # will contain the directories of each class which contains its corresponding training data
validation_class_dirs = []
test_class_dirs = []

for i in range(5):
  os.mkdir(os.path.join(original_dataset_dir, str(i)))
  #os.mkdir(os.path.join(validation_dir, str(i)))
  #os.mkdir(os.path.join(test_dir, str(i)))
  
  train_class_dirs.append(os.path.join(original_dataset_dir, str(i)))
  #validation_class_dirs.append(os.path.join(validation_dir, str(i)))
  #test_class_dirs.append(os.path.join(test_dir, str(i)))
  
  for file in fname_dict[i]:
    file += '.png'
    src = os.path.join(original_dataset_dir, file)
    if not os.path.exists(src):
      continue
    img = load_ben_color(src)
    dst = os.path.join(train_class_dirs[i], file) # copy to train directory
    #shutil.move(src, dst)
    cv2.imwrite(dst, img)
 # do validation as well